In [1]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Using api.load to load the glove-wiki-gigaword-100 pre-trained word2vec model. This specific model is chosen as this model is trained on million of words got from the Wikipedia corpus, this allows us to leverage precomputed word vectors and knowledge that we can fine tune to fit our model.

In [2]:
from gensim import downloader as api
model = api.load("glove-wiki-gigaword-100")




[==================================================] 100.0% 128.1/128.1MB downloaded


read_and_tokenize is a function that takes path of the text file, it then reads it into the variable text using file.read(), then a function from gensim.utils module is used that used to tokenize the text variable into a list of words or tokens which are all lower cased, also punctuations and non alphabetic characters are removed. Finally the tokens extracted are returned.

In [26]:
from gensim.utils import simple_preprocess
def read_and_tokenize(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read()


    tokens = simple_preprocess(text)
    return tokens

tokens = read_and_tokenize('/content/sherlock_padl.txt')

print(tokens)

['the', 'project', 'gutenberg', 'ebook', 'of', 'adventures', 'of', 'sherlock', 'holmes', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www', 'gutenberg', 'org', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'title', 'adventures', 'of', 'sherlock', 'holmes', 'author', 'arthur', 'conan', 'doyle', 'release', 'date', 'february', 'ebook', 'language', 'english', 'credits', 'produced', 'by', 'the', 'online', 'distributed', 'proo

A new word2vec model is intialized, the vector size is chosen as 100 as it has to match the vector size of the pretrained model. for the model, we make it scan through the list of tokens and create a vocabulary of the unique words to be able to build vocabulary. Then all the words present in the vocabulary of the pretrained model is extracted using model.key_to_index.keys, by having update as true we allow that the existing vocabulary is updated, if it was false it will create new ones. Vectors_lockf is an array that is in the training is used in controlling the rate of learning for every word vector, we set it into 1 to make all the word vectors having an equal locking factor of 1 making them 100% updatable. Then for the words that intersect with our vocabulary, we merge their word embeddings from the pretrained model to our new model using intersect_word2vec_format method. Lastly the model is again trained using the new dataset.

In [27]:
from gensim.models import Word2Vec
import numpy as np

new_model = Word2Vec(vector_size=100, min_count=1, epochs=40)
new_model.build_vocab([tokens])

total_examples = new_model.corpus_count

new_model.build_vocab([list(model.key_to_index.keys())], update=True)
new_model.wv.vectors_lockf = np.ones(len(new_model.wv))

new_model.wv.intersect_word2vec_format('~/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz', binary=False, lockf=1.0)

new_model.train([tokens],total_examples = total_examples,epochs = new_model.epochs)





(400000, 4173520)

triplets of words where the middle word is equal "and" is extracted, this is done by looping over the list of the tokens, except the last two tokens so we wont reach out of bounds, for every word, we see if the next index word is equal to "and", if yes, then this word, and the next word, and the word after it are added as a list to the list of triplets.

In [28]:
triplets=[]
for i in range(len(tokens)-2):
  if (tokens[i+1]=='and'):
    triplet = [tokens[i],tokens[i+1],tokens[i+2]]
    triplets.append(triplet)

print(triplets)

[['states', 'and', 'most'], ['cost', 'and', 'with'], ['shut', 'and', 'locked'], ['bye', 'and', 'be'], ['eclipses', 'and', 'predominates'], ['emotions', 'and', 'that'], ['reasoning', 'and', 'observing'], ['gibe', 'and', 'sneer'], ['motives', 'and', 'actions'], ['delicate', 'and', 'finely'], ['his', 'and', 'yet'], ['him', 'and', 'that'], ['dubious', 'and', 'questionable'], ['happiness', 'and', 'the'], ['books', 'and', 'alternating'], ['cocaine', 'and', 'ambition'], ['drug', 'and', 'the'], ['crime', 'and', 'occupied'], ['faculties', 'and', 'extraordinary'], ['clues', 'and', 'clearing'], ['trincomalee', 'and', 'finally'], ['delicately', 'and', 'successfully'], ['friend', 'and', 'companion'], ['wooing', 'and', 'with'], ['again', 'and', 'to'], ['lit', 'and', 'even'], ['chest', 'and', 'his'], ['mood', 'and', 'habit'], ['attitude', 'and', 'manner'], ['dreams', 'and', 'was'], ['bell', 'and', 'was'], ['cigars', 'and', 'indicated'], ['case', 'and', 'gasogene'], ['fire', 'and', 'looked'], ['seven'

we have two functions, the first one takes a word, and then loop over the list of triplets, then for every triplet the last word in the triplet is extracted, and we test if this word share a suffix of length 3 with the word inputted, this is done by using endswith function, if the last word in the triplet share a suffix of length 3 with the inputted 3, then this triplet is appened to a list, at the end this list which contains all the triplets that their last word share a suffix of 3 with the inputted word is returned.

the second function is the same exactly but the last word in the triplets returned from this function just shares a suffix of length 2 with the inputted word.

the second function is constructed because mamost of the words in list L dont have more than 5 triplets that they share a suffix of length 3 with the last word. so this constaint was relaxed a bit to be able to extract a greater number of triplets.

if a word have more than 5 triplets that it share a suffix of length 3 with the last word. then the first function is used, else the second version of the function is used

In [29]:
def filter_triplets_3(word):
  result=[]
  for triplet in triplets:
    last = triplet[2]
    if (last.endswith(word[-3:])):
      result.append(triplet)

  return result

def filter_triplets_2(word):
  result=[]
  for triplet in triplets:
    last = triplet[2]
    if (last.endswith(word[-2:])):
      result.append(triplet)

  return result





for every word in the list L the triplets that its last word share a suffix with the word in L are extracted and saved into a variable.

In [39]:
L = ['gold', 'diamond', 'robbery', 'bank', 'police']

gold_triplets = filter_triplets_2(L[0])
print(len(gold_triplets))

diamond_triplets = filter_triplets_2(L[1])
print(len(diamond_triplets))

robbery_triplets = filter_triplets_3(L[2])
print(len(robbery_triplets))

bank_triplets = filter_triplets_2(L[3])
print(len(bank_triplets))

police_triplets = filter_triplets_2(L[4])
print(len(police_triplets))


40
32
11
8
19


A function that take a list of triplets, then for every triplet, it extracts the first word and third word, and then compute the cosine similarity between the two word embeddings. the cosine similarity is the most common way of measuring the similarity between word vectors. then every triplet with the cosine similarity between the first and third word in the triplet is added to a list and this list is returned at the end when all the triplets are checked.

In [43]:
def triplets_similarity(triplets):

  triplets_sims = []

  for triplet in triplets:

    word1 = triplet[0]
    word3 = triplet[2]

    similarity = new_model.wv.similarity(word1,word3)
    triplets_sims.append([triplet,similarity])

  return triplets_sims



for the lists of triplets of every word in L, the similarity as explained above is calculated, then the built in function in python sorted is used which returns a sorted list from items in an iterable objectm the key set like this to specify the element in which the sorting should be based on, the sorting should be based on the similarity score of every triplet, reverse is set to true to get it in descending order. finally, slicing is used to get the 5 triplets with the highest cosine similarity between their first word and third word. this is done for every triplets list for the words in the list L

In [44]:
gold_similarities = triplets_similarity(gold_triplets)
gold_sorted_similarities = sorted(gold_similarities, key=lambda x: x[1], reverse=True)


print(gold_sorted_similarities[:5])

[[['do', 'and', 'should'], 0.95110756], [['me', 'and', 'could'], 0.89954096], [['at', 'and', 'should'], 0.871415], [['you', 'and', 'would'], 0.86452085], [['matter', 'and', 'should'], 0.8320688]]


In [35]:
diamond_similarities = triplets_similarity(diamond_triplets)
diamond_sorted_similarities = sorted(diamond_similarities, key=lambda x: x[1], reverse=True)

print(diamond_sorted_similarities[:5])

[[['round', 'and', 'round'], 1.0000001], [['up', 'and', 'hand'], 0.90456086], [['up', 'and', 'found'], 0.84162414], [['up', 'and', 'found'], 0.84162414], [['good', 'and', 'kind'], 0.82882684]]


In [36]:
robbery_similarities = triplets_similarity(robbery_triplets)
robbery_sorted_similarities = sorted(robbery_similarities, key=lambda x: x[1], reverse=True)

print(robbery_sorted_similarities[:5])

[[['long', 'and', 'very'], 0.83531034], [['again', 'and', 'very'], 0.8121399], [['day', 'and', 'very'], 0.74541086], [['pay', 'and', 'very'], 0.6459847], [['ten', 'and', 'every'], 0.6184026]]


In [37]:
bank_similarities = triplets_similarity(bank_triplets)
bank_sorted_similarities = sorted(bank_similarities, key=lambda x: x[1], reverse=True)

print(bank_sorted_similarities[:5])

[[['me', 'and', 'think'], 0.83854896], [['done', 'and', 'think'], 0.8142789], [['loves', 'and', 'think'], 0.6173758], [['do', 'and', 'frank'], 0.38277626], [['handy', 'and', 'think'], 0.27069616]]


In [42]:
police_similarities = triplets_similarity(police_triplets)
police_sorted_similarities = sorted(police_similarities, key=lambda x: x[1], reverse=True)

print(police_sorted_similarities[:5])



[[['head', 'and', 'face'], 0.8196561], [['men', 'and', 'once'], 0.7118795], [['novel', 'and', 'romance'], 0.6675822], [['holmes', 'and', 'since'], 0.66330093], [['help', 'and', 'advice'], 0.59972864]]
